In [ ]:
pip install gspread pandas


In [ ]:
!pip install flask_talisman pyngrok

In [ ]:
!pip install oauth2client

In [ ]:
!pip install gspread google-api-python-client oauth2client

In [ ]:
!pandas numpy dash plotly

/bin/bash: line 1: pandas: command not found


In [ ]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
!pip install dash plotly

In [ ]:
import dash
import plotly

print("Dash version:", dash.__version__)
print("Plotly version:", plotly.__version__)

Dash version: 2.18.2
Plotly version: 5.24.1


In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 폴더 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 분석: 관리지사명, KTT월정료, 계약번호 등 요약
def summarize_data(df):
    df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce').fillna(0)  # 숫자 변환
    df['계약번호'] = pd.to_numeric(df['계약번호'], errors='coerce').fillna(0)
    summary = df.groupby('관리지사명').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()
    summary['월정료합계'] = summary['월정료합계'].apply(lambda x: f"{x:,.0f}")  # 천 단위 표시
    return summary

# 4. Dash 대시보드 생성
def create_dash_app(df):
    app = Dash(__name__)

    # 초기 차트 생성
    def create_charts(data, group_by, value_col, chart_type="bar"):
        if chart_type == "bar":
            fig = px.bar(data, x=group_by, y=value_col, title=f'{group_by} 별 {value_col} 분포도', text=value_col)
        elif chart_type == "pie":
            fig = px.pie(data, values=value_col, names=group_by, title=f'{group_by} 별 {value_col} 비율')
        else:
            fig = px.line(data, x=group_by, y=value_col, title=f'{group_by} 별 {value_col} 분포도')
        return fig

    # 초기 데이터 요약
    summary = summarize_data(df)
    bar_chart = create_charts(summary, '관리지사명', '건수', chart_type="bar")
    pie_chart = create_charts(summary, '관리지사명', '월정료합계', chart_type="pie")

    # 대시보드 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명별 건수 분포도"),
            dcc.Graph(id='bar-chart', figure=bar_chart)
        ]),
        html.Div([
            html.Label("관리지사명별 월정료 분포도"),
            dcc.Graph(id='pie-chart', figure=pie_chart)
        ]),
        html.Div([
            html.Label("데이터 필터링:"),
            dcc.Dropdown(
                id='group-by',
                options=[
                    {'label': '관리지사명', 'value': '관리지사명'},
                    {'label': '영업구역정보', 'value': '영업구역정보'},
                    {'label': '서비스(소)', 'value': '서비스(소)'}
                ],
                value='관리지사명'
            ),
            dcc.Dropdown(
                id='chart-type',
                options=[
                    {'label': '막대형 차트', 'value': 'bar'},
                    {'label': '원형 차트', 'value': 'pie'},
                    {'label': '선형 차트', 'value': 'line'}
                ],
                value='bar'
            )
        ]),
        html.Div([
            dcc.Graph(id='dynamic-chart')
        ])
    ])

    # 데이터 새로고침 콜백
    @app.callback(
        Output('dynamic-chart', 'figure'),
        [Input('group-by', 'value'), Input('chart-type', 'value')]
    )
    def update_charts(group_by, chart_type):
        if group_by not in df.columns:
            return px.bar(title="데이터 없음")
        updated_summary = df.groupby(group_by).agg(건수=('계약번호', 'count')).reset_index()
        return create_charts(updated_summary, group_by, '건수', chart_type)

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")
    print(df.head())

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(df)
    app.run_server(debug=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Sheets에서 db 시트 데이터를 가져오는 중...


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 폴더 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 전처리: "지사" 숨기기 및 사용자 정의 정렬
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환 (₩, 쉼표 제거)
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양','의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return summary

# 4. Dash 대시보드 생성
def create_dash_app(summary_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # Dash 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명_수정'].unique()],
                value=None,
                multi=False,
                placeholder="관리지사를 선택하세요"
            ),
        ], style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='branch-chart'),
        ]),
        html.Div(id='region-section')
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 관리지사 그래프 및 영업구역정보 동적 업데이트
    @app.callback(
        [Output('branch-chart', 'figure'), Output('region-section', 'children')],
        [Input('branch-filter', 'value')]
    )
    def update_charts(selected_branch):
        filtered_df = summary_df[summary_df['관리지사명_수정'] == selected_branch] if selected_branch else summary_df

        # 관리지사별 그래프
        branch_chart = px.bar(
            filtered_df.groupby('관리지사명_수정', as_index=False).agg(
                건수=('건수', 'sum'),
                월정료합계=('월정료합계', 'sum')
            ),
            x='관리지사명_수정',
            y='건수',
            text='월정료합계',
            title=f"{selected_branch} 관리지사 건수 및 월정료 분포도" if selected_branch else "전체 관리지사 건수 및 월정료 분포도",
            color='관리지사명_수정'
        )
        branch_chart.update_traces(texttemplate='%{text:.2s}천원', textposition='outside')
        branch_chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            xaxis_title="관리지사명",
            yaxis_title="건수",
            font={'family': 'Arial, sans-serif', 'size': 14},
            bargap=0.2
        )

        # 영업구역정보 그래프 (선택된 관리지사일 경우만)
        if selected_branch:
            region_chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='표시값',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료 분포도",
                color='영업구역정보'
            )
            region_chart.update_traces(texttemplate='%{text}', textposition='outside')
            region_chart.update_layout(
                title={'font': {'size': 20}, 'x': 0.5},
                xaxis_title="영업구역정보",
                yaxis_title="건수",
                font={'family': 'Arial, sans-serif', 'size': 14},
                bargap=0.2
            )

            region_section = html.Div([
                html.H2(f"{selected_branch} 영업구역정보별 차트", style={'textAlign': 'center'}),
                dcc.Graph(figure=region_chart)
            ])
        else:
            region_section = html.Div()

        return branch_chart, region_section

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")

    # 데이터 전처리 및 사용자 정의 정렬 적용
    processed_summary_df = preprocess_data(raw_df)
    print(processed_summary_df)

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(processed_summary_df)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Sheets에서 db 시트 데이터를 가져오는 중...


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 폴더 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(credentials=creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 전처리: "지사" 숨기기 및 사용자 정의 정렬
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환 (₩, 쉼표 제거)
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return summary

# 4. Dash 대시보드 생성
def create_dash_app(summary_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # Dash 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명_수정'].unique()],
                value=None,
                multi=False,
                placeholder="관리지사를 선택하세요"
            ),
        ], style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='branch-chart'),
        ]),
        html.Div(id='region-section')
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 관리지사 그래프 및 영업구역정보 동적 업데이트
    @app.callback(
        [Output('branch-chart', 'figure'), Output('region-section', 'children')],
        [Input('branch-filter', 'value')]
    )
    def update_charts(selected_branch):
        if selected_branch:
            filtered_df = summary_df[summary_df['관리지사명_수정'] == selected_branch]
        else:
            filtered_df = summary_df

        # 관리지사별 그래프
        branch_chart = px.bar(
            filtered_df.groupby('관리지사명_수정', as_index=False).agg(
                건수=('건수', 'sum'),
                월정료합계=('월정료합계', 'sum')
            ),
            x='관리지사명_수정',
            y='건수',
            text='건수',
            title=f"{selected_branch} 관리지사 건수 및 월정료 분포도" if selected_branch else "전체 관리지사 건수 및 월정료 분포도",
            color='관리지사명_수정'
        )
        branch_chart.update_traces(textposition='inside')
        branch_chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            xaxis_title="관리지사명",
            yaxis_title="건수",
            font={'family': 'Arial, sans-serif', 'size': 14},
            bargap=0.2,
            uniformtext_minsize=12,  # 텍스트 크기 조정
            uniformtext_mode='hide'  # 텍스트 잘림 방지
        )

        # 영업구역정보 그래프 (선택된 관리지사일 경우만)
        if selected_branch:
            region_chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='표시값',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료 분포도",
                color='영업구역정보'
            )
            region_chart.update_traces(texttemplate='%{text}', textposition='outside')
            region_chart.update_layout(
                title={'font': {'size': 20}, 'x': 0.5},
                xaxis_title="영업구역정보",
                yaxis_title="건수",
                font={'family': 'Arial, sans-serif', 'size': 14},
                bargap=0.2
            )

            region_section = html.Div([
                html.H2(f"{selected_branch} 영업구역정보별 차트", style={'textAlign': 'center'}),
                dcc.Graph(figure=region_chart)
            ])
        else:
            region_section = html.Div()

        return branch_chart, region_section

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")

    # 데이터 전처리 및 사용자 정의 정렬 적용
    processed_summary_df = preprocess_data(raw_df)
    print(processed_summary_df)

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(processed_summary_df)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Sheets에서 db 시트 데이터를 가져오는 중...


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 폴더 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(credentials=creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, cleient):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 전처리: "지사" 숨기기 및 사용자 정의 정렬
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환 (₩, 쉼표 제거)
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 점유율 계산
    total_revenue = summary['월정료합계'].sum()
    summary['월정료점유율'] = (summary['월정료합계'] / total_revenue * 100).round(2)

    # 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['건수']}건\n{int(row['월정료합계']):,}천원 ({row['월정료점유율']}%)", axis=1
    )
    return summary

# 4. Dash 대시보드 생성
def create_dash_app(summary_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # Dash 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명_수정'].unique()],
                value=None,
                multi=False,
                placeholder="관리지사를 선택하세요"
            ),
        ], style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='region-chart')
        ]),
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 영업구역정보 그래프 업데이트
    @app.callback(
        Output('region-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_region_chart(selected_branch):
        if selected_branch:
            filtered_df = summary_df[summary_df['관리지사명_수정'] == selected_branch]
        else:
            filtered_df = summary_df

        # 영업구역정보 그래프
        region_chart = px.bar(
            filtered_df,
            x='영업구역정보',
            y='건수',
            text='건수',
            color='영업구역정보',
            title=f"{selected_branch} 영업구역정보별 건수 및 월정료 분포도" if selected_branch else "영업구역정보별 건수 및 월정료 분포도"
        )
        region_chart.update_traces(
            textposition='inside',  # 건수는 그래프 안쪽
            hovertemplate=(
                "영업구역정보: %{x}<br>" +
                "건수: %{y}건<br>" +
                "월정료: %{text}<br>"
            )
        )
        region_chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            xaxis_title="영업구역정보",
            yaxis_title="건수",
            font={'family': 'Arial, sans-serif', 'size': 14},
            bargap=0.2,
            uniformtext_minsize=12,  # 텍스트 크기 조정
            uniformtext_mode='hide'  # 텍스트 잘림 방지
        )

        return region_chart

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")

    # 데이터 전처리 및 사용자 정의 정렬 적용
    processed_summary_df = preprocess_data(raw_df)
    print(processed_summary_df)

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(processed_summary_df)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Sheets에서 db 시트 데이터를 가져오는 중...


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 폴더 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(credentials=creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 전처리: "지사" 숨기기 및 사용자 정의 정렬
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환 (₩, 쉼표 제거)
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# 4. Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # Dash 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                multi=False,
                placeholder="관리지사를 선택하세요"
            ),
        ], style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='main-chart')
        ]),
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 차트 업데이트 (지사별 또는 영업구역별)
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 영업구역정보별 그래프 표시
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료 분포도"
            )
        else:
            # 지사별 그래프 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="지사별 건수 및 월정료 분포도"
            )

        chart.update_traces(
            textposition='inside',  # 건수는 그래프 내부
            hovertemplate=(
                "건수: %{y}건<br>" +
                "월정료: %{text}<br>"
            )
        )
        chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            font={'family': 'Arial, sans-serif', 'size': 14},
            bargap=0.2,
            uniformtext_minsize=12,  # 텍스트 크기 조정
            uniformtext_mode='hide'  # 텍스트 잘림 방지
        )

        return chart

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")

    # 데이터 전처리 및 사용자 정의 정렬 적용
    branch_summary, region_summary = preprocess_data(raw_df)

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(branch_summary, region_summary)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Sheets에서 db 시트 데이터를 가져오는 중...


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 폴더 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(credentials=creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 전처리: "지사" 숨기기 및 사용자 정의 정렬
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환 (₩, 쉼표 제거)
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 점유율 계산
    branch_summary['건수점유율'] = (branch_summary['건수'] / branch_summary['건수'].sum() * 100).round(2)
    branch_summary['월정료점유율'] = (branch_summary['월정료합계'] / branch_summary['월정료합계'].sum() * 100).round(2)

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# 4. Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # Dash 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                multi=False,
                placeholder="관리지사를 선택하세요"
            ),
        ], style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='main-chart')
        ]),
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 차트 업데이트 (지사별 또는 영업구역별)
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 영업구역정보별 그래프 표시
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료 분포도"
            )
            chart.update_traces(
                textposition='inside',  # 건수는 그래프 내부
                hovertemplate=(
                    "영업구역정보: %{x}<br>" +
                    "건수: %{y}건<br>" +
                    "월정료: %{text}<br>"
                )
            )
        else:
            # 지사별 그래프 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="지사별 건수 및 월정료 분포도"
            )
            chart.update_traces(
                texttemplate=(
                    "건수: %{y}건<br>" +
                    "월정료: %{customdata[0]:,.0f}천원 (%{customdata[1]:.2f}%)"
                ),
                customdata=branch_summary[['월정료합계', '월정료점유율']].values,
                textposition='inside'
            )

        chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            font={'family': 'Arial, sans-serif', 'size': 14},
            bargap=0.2,
            uniformtext_minsize=12,  # 텍스트 크기 조정
            uniformtext_mode='hide'  # 텍스트 잘림 방지
        )

        return chart

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")

    # 데이터 전처리 및 사용자 정의 정렬 적용
    branch_summary, region_summary = preprocess_data(raw_df)

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(branch_summary, region_summary)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Sheets에서 db 시트 데이터를 가져오는 중...


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 폴더 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(credentials=creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 전처리: "지사" 숨기기 및 사용자 정의 정렬
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환 (₩, 쉼표 제거)
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# 4. Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # Dash 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                multi=False,
                placeholder="관리지사를 선택하세요"
            ),
        ], style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='main-chart')
        ]),
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 차트 업데이트 (지사별 또는 영업구역별)
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 영업구역정보별 그래프 표시
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(
                textposition='inside',
                hovertemplate=(
                    "영업구역정보: %{x}<br>" +
                    "건수: %{y}건<br>" +
                    "월정료: %{customdata[0]:,.0f}천원<br>"
                ),
                customdata=filtered_df[['월정료합계']].values
            )
        else:
            # 지사별 그래프 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="지사별 건수 및 월정료"
            )
            chart.update_traces(
                textposition='inside',
                hovertemplate=(
                    "지사: %{x}<br>" +
                    "건수: %{y}건<br>" +
                    "월정료: %{customdata[0]:,.0f}천원<br>"
                ),
                customdata=branch_summary[['월정료합계']].values
            )

        chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            font={'family': 'Arial, sans-serif', 'size': 14},
            bargap=0.2,
            uniformtext_minsize=12,
            uniformtext_mode='hide'
        )

        return chart

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")

    # 데이터 전처리 및 사용자 정의 정렬 적용
    branch_summary, region_summary = preprocess_data(raw_df)

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(branch_summary, region_summary)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Sheets에서 db 시트 데이터를 가져오는 중...


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 폴더 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(credentials=creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환 (₩, 쉼표 제거)
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# 4. Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # Dash 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                multi=False,
                placeholder="관리지사를 선택하세요"
            ),
        ], style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='main-chart')
        ]),
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 차트 업데이트 (지사별 또는 영업구역별)
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 영업구역정보별 그래프 표시
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(
                textposition='inside',
                hovertemplate=(
                    "영업구역정보: %{x}<br>" +
                    "건수: %{y}건<br>" +
                    "월정료: %{customdata[0]:,.0f}천원<br>"
                ),
                customdata=filtered_df[['월정료합계']].values
            )
        else:
            # 지사별 그래프 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="지사별 건수 및 월정료"
            )
            chart.update_traces(
                textposition='inside',
                hovertemplate=(
                    "지사: %{x}<br>" +
                    "건수: %{y}건<br>" +
                    "월정료: %{customdata[0]:,.0f}천원<br>"
                ),
                customdata=branch_summary[['월정료합계']].values
            )

        chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            font={'family': 'Arial, sans-serif', 'size': 14},
            bargap=0.2,
            uniformtext_minsize=12,
            uniformtext_mode='hide'
        )

        return chart

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")

    # 데이터 전처리 및 사용자 정의 정렬 적용
    branch_summary, region_summary = preprocess_data(raw_df)

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(branch_summary, region_summary)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Sheets에서 db 시트 데이터를 가져오는 중...


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환 (₩, 쉼표 제거)
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# 4. Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # Dash 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                multi=False,
                placeholder="관리지사를 선택하세요"
            ),
        ], style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='main-chart')
        ]),
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 차트 업데이트 (지사별 또는 영업구역별)
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 영업구역정보별 그래프 표시
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(
                textposition='inside',
                hovertemplate=(
                    "영업구역정보: %{x}<br>" +
                    "건수: %{y}건<br>" +
                    "월정료: %{customdata[0]:,.0f}천원<br>"
                ),
                customdata=filtered_df[['월정료합계']].values
            )
        else:
            # 지사별 그래프 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="지사별 건수 및 월정료"
            )
            chart.update_traces(
                textposition='inside',
                hovertemplate=(
                    "지사: %{x}<br>" +
                    "건수: %{y}건<br>" +
                    "월정료: %{customdata[0]:,.0f}천원<br>"
                ),
                customdata=branch_summary[['월정료합계']].values
            )

        chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            font={'family': 'Arial, sans-serif', 'size': 14},
            bargap=0.2,
            uniformtext_minsize=12,
            uniformtext_mode='hide'
        )

        return chart

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")

    # 데이터 전처리 및 사용자 정의 정렬 적용
    branch_summary, region_summary = preprocess_data(raw_df)

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(branch_summary, region_summary)

    # 동적으로 고유한 포트를 생성
    port = random.randint(1024, 65535)
    app.run_server(debug=True, port=port)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Sheets에서 db 시트 데이터를 가져오는 중...


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # 인증 키 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # 월정료 데이터를 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:"),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='main-chart')
    ])

    # 콜백 정의
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 해당 영업구역정보별 데이터 표시
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(
                textposition='inside',
                hovertemplate=(
                    "영업구역정보: %{x}<br>" +
                    "건수: %{y}건<br>" +
                    "월정료: %{customdata[0]:,.0f}천원"
                ),
                customdata=filtered_df[['월정료합계']].values
            )
        else:
            # 지사별 데이터 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="지사별 건수 및 월정료"
            )
            chart.update_traces(
                textposition='inside',
                hovertemplate=(
                    "지사: %{x}<br>" +
                    "건수: %{y}건<br>" +
                    "월정료: %{customdata[0]:,.0f}천원"
                ),
                customdata=branch_summary[['월정료합계']].values
            )

        chart.update_layout(
            title={'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            font=dict(size=14),
            uniformtext_minsize=8,
            uniformtext_mode='hide'
        )
        return chart

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # 데이터 가져오기
    raw_data = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리
    branch_summary, region_summary = preprocess_data(raw_data)

    # 대시보드 실행
    app = create_dash_app(branch_summary, region_summary)
    app.run_server(debug=True, use_reloader=False)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # 인증 키 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # 월정료 데이터를 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:"),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='main-chart')
    ])

    # 콜백 정의
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 해당 영업구역정보별 데이터 표시
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(
                texttemplate="건수: %{y}\n월정료: %{customdata[0]:,.0f}천원",
                customdata=filtered_df[['월정료합계']].values,
                textposition='outside'
            )
        else:
            # 지사별 데이터 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="지사별 건수 및 월정료"
            )
            chart.update_traces(
                texttemplate="건수: %{y}\n월정료: %{customdata[0]:,.0f}천원",
                customdata=branch_summary[['월정료합계']].values,
                textposition='outside'
            )

        chart.update_layout(
            title={'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            xaxis=dict(tickangle=-45),  # X축 라벨 각도 조정
            font=dict(size=14),
            uniformtext_minsize=8,
            uniformtext_mode='hide'
        )
        return chart

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # 데이터 가져오기
    raw_data = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리
    branch_summary, region_summary = preprocess_data(raw_data)

    # 대시보드 실행
    app = create_dash_app(branch_summary, region_summary)
    app.run_server(debug=True, use_reloader=False)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # 인증 키 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # 월정료 데이터를 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:"),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='main-chart')
    ])

    # 콜백 정의
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 해당 영업구역정보 필터링
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            # 중앙지사에 해당하는 영업구역만 필터링
            if selected_branch == '중앙':
                filtered_df = filtered_df[filtered_df['영업구역정보'].isin(
                    ['G00401', 'G00402', 'G00403', 'G00404', 'G00405', 'G00406', 'G00407']
                )]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(
                texttemplate="건수: %{y}\n월정료: %{customdata[0]:,.0f}천원",
                customdata=filtered_df[['월정료합계']].values,
                textposition='outside'
            )
        else:
            # 지사별 데이터 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="지사별 건수 및 월정료"
            )
            chart.update_traces(
                texttemplate="건수: %{y}\n월정료: %{customdata[0]:,.0f}천원",
                customdata=branch_summary[['월정료합계']].values,
                textposition='outside'
            )

        chart.update_layout(
            title={'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            xaxis=dict(tickangle=-45),  # X축 라벨 각도 조정
            font=dict(size=14),
            uniformtext_minsize=8,
            uniformtext_mode='hide'
        )
        return chart

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # 데이터 가져오기
    raw_data = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리
    branch_summary, region_summary = preprocess_data(raw_data)

    # 대시보드 실행
    app = create_dash_app(branch_summary, region_summary)
    app.run_server(debug=True, use_reloader=False)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # 인증 키 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # 월정료 데이터를 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:"),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='main-chart')
    ])

    # 콜백 정의
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 해당 영업구역정보 필터링
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            # 중앙지사에 해당하는 영업구역만 필터링
            if selected_branch == '중앙':
                filtered_df = filtered_df[filtered_df['영업구역정보'].isin(
                    ['G00401', 'G00402', 'G00403', 'G00404', 'G00405', 'G00406', 'G00407']
                )]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(
                texttemplate="건수: %{y}\n월정료: %{customdata[0]:,.0f}천원",
                customdata=filtered_df[['월정료합계']].values,
                textposition='outside'
            )
        else:
            # 지사별 데이터 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="전체 관리지사 건수 및 월정료 분포도"
            )
            chart.update_traces(
                texttemplate="건수: %{y}\n월정료: %{customdata[0]:,.0f}천원",
                customdata=branch_summary[['월정료합계']].values,
                textposition='outside'
            )

        chart.update_layout(
            title={'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            xaxis=dict(tickangle=-45),  # X축 라벨 각도 조정
            font=dict(size=14),
            uniformtext_minsize=8,
            uniformtext_mode='hide'
        )
        return chart

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # 데이터 가져오기
    raw_data = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리
    branch_summary, region_summary = preprocess_data(raw_data)

    # 대시보드 실행
    app = create_dash_app(branch_summary, region_summary)
    app.run_server(debug=True, use_reloader=False)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # 인증 키 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # 월정료 데이터를 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 지사별 데이터 요약
    branch_summary = df.groupby('관리지사명_수정').agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 영업구역정보별 데이터 요약
    region_summary = df.groupby(['관리지사명_수정', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    return branch_summary, region_summary

# Dash 대시보드 생성
def create_dash_app(branch_summary, region_summary):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:"),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명_수정'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='main-chart')
    ])

    # 콜백 정의
    @app.callback(
        Output('main-chart', 'figure'),
        [Input('branch-filter', 'value')]
    )
    def update_chart(selected_branch):
        if selected_branch:
            # 특정 지사 선택 시 해당 영업구역정보 필터링
            filtered_df = region_summary[region_summary['관리지사명_수정'] == selected_branch]
            # 중앙지사에 해당하는 영업구역만 필터링
            if selected_branch == '중앙':
                filtered_df = filtered_df[filtered_df['영업구역정보'].isin(
                    ['G00401', 'G00402', 'G00403', 'G00404', 'G00405', 'G00406', 'G00407']
                )]
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='건수',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(
                texttemplate="건수: %{y}\n월정료: %{customdata[0]:,.0f}천원",
                customdata=filtered_df[['월정료합계']].values,
                textposition='outside'
            )
        else:
            # 지사별 데이터 표시
            chart = px.bar(
                branch_summary,
                x='관리지사명_수정',
                y='건수',
                text='건수',
                color='관리지사명_수정',
                title="전체 관리지사 건수 및 월정료 분포도"
            )
            chart.update_traces(
                texttemplate="건수: %{y}\n월정료: %{customdata[0]:,.0f}천원",
                customdata=branch_summary[['월정료합계']].values,
                textposition='outside'
            )

        chart.update_layout(
            title={'x': 0.5},
            xaxis_title="",
            yaxis_title="건수",
            xaxis=dict(tickangle=-45),  # X축 라벨 각도 조정
            font=dict(size=14),
            uniformtext_minsize=8,
            uniformtext_mode='hide'
        )
        return chart

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # 데이터 가져오기
    raw_data = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리
    branch_summary, region_summary = preprocess_data(raw_data)

    # 대시보드 실행
    app = create_dash_app(branch_summary, region_summary)
    app.run_server(debug=True, use_reloader=False)

In [ ]:
# 구역별성공

import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# 1. Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 폴더 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# 2. Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 3. 데이터 전처리: KTT월정료 변환 및 사용자 정의 정렬
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환 (₩, 쉼표 제거)
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 사용자 정의 정렬 순서 설정
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '원주', '강릉']
    df['관리지사명_정렬'] = df['관리지사명'].str.replace('지사', '')  # "지사" 제외
    df['관리지사명_정렬'] = pd.Categorical(df['관리지사명_정렬'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_정렬')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return summary


# 4. Dash 대시보드 생성
def create_dash_app(summary_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # Dash 레이아웃
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명'].unique()],
                value=None,
                multi=False,
                placeholder="관리지사를 선택하세요"
            ),
        ], style={'margin-bottom': '20px'}),
        html.Div([
            dcc.Graph(id='branch-chart'),
        ]),
        html.Div(id='region-section')
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 관리지사 그래프 및 영업구역정보 동적 업데이트
    @app.callback(
        [Output('branch-chart', 'figure'), Output('region-section', 'children')],
        [Input('branch-filter', 'value')]
    )
    def update_charts(selected_branch):
        filtered_df = summary_df[summary_df['관리지사명'] == selected_branch] if selected_branch else summary_df

        # 관리지사별 그래프
        branch_chart = px.bar(
            filtered_df.groupby('관리지사명', as_index=False).agg(
                건수=('건수', 'sum'),
                월정료합계=('월정료합계', 'sum')
            ),
            x='관리지사명',
            y='건수',
            text='월정료합계',
            title=f"{selected_branch} 관리지사 건수 및 월정료 분포도" if selected_branch else "전체 관리지사 건수 및 월정료 분포도",
            color='관리지사명'
        )
        branch_chart.update_traces(texttemplate='%{text:.2s}천원', textposition='outside')
        branch_chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            xaxis_title="관리지사명",
            yaxis_title="건수",
            font={'family': 'Arial, sans-serif', 'size': 14},
            bargap=0.2
        )

        # 영업구역정보 그래프 (선택된 관리지사일 경우만)
        if selected_branch:
            region_chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='표시값',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료 분포도",
                color='영업구역정보'
            )
            region_chart.update_traces(texttemplate='%{text}', textposition='outside')
            region_chart.update_layout(
                title={'font': {'size': 20}, 'x': 0.5},
                xaxis_title="영업구역정보",
                yaxis_title="건수",
                font={'family': 'Arial, sans-serif', 'size': 14},
                bargap=0.2
            )

            region_section = html.Div([
                html.H2(f"{selected_branch} 영업구역정보별 차트", style={'textAlign': 'center'}),
                dcc.Graph(figure=region_chart)
            ])
        else:
            region_section = html.Div()

        return branch_chart, region_section

    return app

# 5. 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    print(f"Google Sheets에서 {sheet_name} 시트 데이터를 가져오는 중...")
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)
    print("데이터 가져오기 완료!")

    # 데이터 전처리 및 사용자 정의 정렬 적용
    processed_summary_df = preprocess_data(raw_df)
    print(processed_summary_df)

    # Dash 대시보드 실행
    print("대시보드 실행 중...")
    app = create_dash_app(processed_summary_df)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거 및 사용자 정의 정렬
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return summary

# Dash 대시보드 생성
def create_dash_app(summary_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
            html.Label("차트 유형:", style={'font-weight': 'bold', 'margin-top': '10px'}),
            dcc.Dropdown(
                id='chart-type',
                options=[
                    {'label': '막대형 차트', 'value': 'bar'},
                    {'label': '원형 차트', 'value': 'pie'},
                    {'label': '선형 차트', 'value': 'line'}
                ],
                value='bar',
                placeholder="차트 유형을 선택하세요"
            )
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='dynamic-chart'),
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 콜백 정의
    @app.callback(
        Output('dynamic-chart', 'figure'),
        [Input('branch-filter', 'value'), Input('chart-type', 'value')]
    )
    def update_chart(selected_branch, chart_type):
        # 필터링된 데이터프레임 생성
        filtered_df = summary_df[summary_df['관리지사명'] == selected_branch] if selected_branch else summary_df

        # 차트 유형에 따라 그래프 생성
        if chart_type == 'bar':
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='표시값',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료" if selected_branch else "전체 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(textposition='outside')
        elif chart_type == 'pie':
            chart = px.pie(
                filtered_df,
                values='건수',
                names='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 비율" if selected_branch else "전체 영업구역정보별 비율",
                hole=0.4
            )
        elif chart_type == 'line':
            chart = px.line(
                filtered_df,
                x='영업구역정보',
                y='건수',
                title=f"{selected_branch} 영업구역정보별 건수 추세" if selected_branch else "전체 영업구역정보별 건수 추세",
                markers=True
            )
            chart.update_traces(text='표시값')

        # 그래프 레이아웃 업데이트
        chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            font={'family': 'Arial, sans-serif', 'size': 14},
            xaxis=dict(tickangle=-45),
            yaxis_title="건수" if chart_type in ['bar', 'line'] else None
        )
        return chart

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리 및 사용자 정의 정렬 적용
    processed_summary_df = preprocess_data(raw_df)

    # Dash 대시보드 실행
    app = create_dash_app(processed_summary_df)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)


In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os
import random

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거 및 사용자 정의 정렬
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return summary

# Dash 대시보드 생성
def create_dash_app(summary_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
            html.Label("차트 유형:", style={'font-weight': 'bold', 'margin-top': '10px'}),
            dcc.Dropdown(
                id='chart-type',
                options=[
                    {'label': '막대형 차트', 'value': 'bar'},
                    {'label': '원형 차트', 'value': 'pie'},
                    {'label': '선형 차트', 'value': 'line'}
                ],
                value='bar',
                placeholder="차트 유형을 선택하세요"
            )
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='dynamic-chart'),
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 콜백 정의
    @app.callback(
        Output('dynamic-chart', 'figure'),
        [Input('branch-filter', 'value'), Input('chart-type', 'value')]
    )
    def update_chart(selected_branch, chart_type):
        # 필터링된 데이터프레임 생성
        filtered_df = summary_df[summary_df['관리지사명'] == selected_branch] if selected_branch else summary_df

        # 데이터가 비어있는 경우 기본 메시지 반환
        if filtered_df.empty:
            return px.bar(
                title="데이터가 없습니다",
                xaxis_title="영업구역정보",
                yaxis_title="건수"
            )

        # 차트 유형에 따라 그래프 생성
        if chart_type == 'bar':
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='표시값',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료" if selected_branch else "전체 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(textposition='outside')
        elif chart_type == 'pie':
            chart = px.pie(
                filtered_df,
                values='건수',
                names='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 비율" if selected_branch else "전체 영업구역정보별 비율",
                hole=0.4
            )
        elif chart_type == 'line':
            chart = px.line(
                filtered_df,
                x='영업구역정보',
                y='건수',
                title=f"{selected_branch} 영업구역정보별 건수 추세" if selected_branch else "전체 영업구역정보별 건수 추세",
                markers=True
            )
            chart.update_traces(text='표시값')

        # 그래프 레이아웃 업데이트
        chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            font={'family': 'Arial, sans-serif', 'size': 14},
            xaxis=dict(tickangle=-45),
            yaxis_title="건수" if chart_type in ['bar', 'line'] else None
        )
        return chart

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리 및 사용자 정의 정렬 적용
    processed_summary_df = preprocess_data(raw_df)

    # Dash 대시보드 실행
    app = create_dash_app(processed_summary_df)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)


In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import os
import random

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거 및 사용자 정의 정렬
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 점유율 계산
    total_count = df['계약번호'].count()
    total_monthly_fee = df['KTT월정료'].sum()
    df['건수점유율(%)'] = (df.groupby('관리지사명')['계약번호'].transform('count') / total_count * 100).round(2)
    df['월정료점유율(%)'] = (df.groupby('관리지사명')['KTT월정료'].transform('sum') / total_monthly_fee * 100).round(2)

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명', '영업구역정보']).agg(
        건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum'),
        건수점유율=('건수점유율(%)', 'mean'),
        월정료점유율=('월정료점유율(%)', 'mean')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return df, summary

# Dash 대시보드 생성
def create_dash_app(summary_df, raw_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
            html.Label("차트 유형:", style={'font-weight': 'bold', 'margin-top': '10px'}),
            dcc.Dropdown(
                id='chart-type',
                options=[
                    {'label': '막대형 차트', 'value': 'bar'},
                    {'label': '원형 차트', 'value': 'pie'},
                    {'label': '선형 차트', 'value': 'line'}
                ],
                value='bar',
                placeholder="차트 유형을 선택하세요"
            )
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='dynamic-chart'),
        html.H2("현황표", style={'textAlign': 'center', 'margin-top': '20px'}),
        dash_table.DataTable(
            id='data-table',
            columns=[
                {"name": i, "id": i} for i in ["관리지사명", "영업구역정보", "건수", "KTT월정료", "건수점유율(%)", "월정료점유율(%)"]
            ],
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
            style_data={'border': '1px solid grey'},
            page_size=10,
            style_as_list_view=True,
            data=raw_df.to_dict('records')
        )
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 콜백 정의
    @app.callback(
        Output('dynamic-chart', 'figure'),
        [Input('branch-filter', 'value'), Input('chart-type', 'value')]
    )
    def update_chart(selected_branch, chart_type):
        # 필터링된 데이터프레임 생성
        filtered_df = summary_df[summary_df['관리지사명'] == selected_branch] if selected_branch else summary_df

        # 데이터가 비어있는 경우 기본 메시지 반환
        if filtered_df.empty:
            return px.bar(
                title="데이터가 없습니다",
                xaxis_title="영업구역정보",
                yaxis_title="건수"
            )

        # 차트 유형에 따라 그래프 생성
        if chart_type == 'bar':
            chart = px.bar(
                filtered_df,
                x='영업구역정보',
                y='건수',
                text='표시값',
                color='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 건수 및 월정료" if selected_branch else "전체 영업구역정보별 건수 및 월정료"
            )
            chart.update_traces(textposition='outside')
        elif chart_type == 'pie':
            chart = px.pie(
                filtered_df,
                values='건수',
                names='영업구역정보',
                title=f"{selected_branch} 영업구역정보별 비율" if selected_branch else "전체 영업구역정보별 비율",
                hole=0.4
            )
        elif chart_type == 'line':
            chart = px.line(
                filtered_df,
                x='영업구역정보',
                y='건수',
                title=f"{selected_branch} 영업구역정보별 건수 추세" if selected_branch else "전체 영업구역정보별 건수 추세",
                markers=True
            )
            chart.update_traces(text='표시값')

        # 그래프 레이아웃 업데이트
        chart.update_layout(
            title={'font': {'size': 20}, 'x': 0.5},
            font={'family': 'Arial, sans-serif', 'size': 14},
            xaxis=dict(tickangle=-45),
            yaxis_title="건수" if chart_type in ['bar', 'line'] else None
        )
        return chart

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리 및 사용자 정의 정렬 적용
    raw_df, processed_summary_df = preprocess_data(raw_df)

    # Dash 대시보드 실행
    app = create_dash_app(processed_summary_df, raw_df)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import os
import random

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거 및 사용자 정의 정렬
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명']).agg(
        계약번호_건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['계약번호_건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return df, summary

# Dash 대시보드 생성
def create_dash_app(summary_df, raw_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
            html.Label("영업구역정보 필터:", style={'font-weight': 'bold', 'margin-top': '10px'}),
            dcc.Dropdown(
                id='region-filter',
                placeholder="영업구역명을 선택하세요",
                multi=False
            )
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='main-chart'),
        html.H2("현황표", style={'textAlign': 'center', 'margin-top': '20px'}),
        dash_table.DataTable(
            id='data-table',
            columns=[
                {"name": "관리지사명", "id": "관리지사명"},
                {"name": "영업구역정보", "id": "영업구역정보"},
                {"name": "계약번호_건수", "id": "계약번호_건수"},
                {"name": "KTT월정료", "id": "KTT월정료"}
            ],
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
            style_data={'border': '1px solid grey'},
            page_size=10,
            style_as_list_view=True
        )
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 콜백: 메인 차트 및 테이블 업데이트
    @app.callback(
        [Output('main-chart', 'figure'),
         Output('data-table', 'data'),
         Output('region-filter', 'options')],
        [Input('branch-filter', 'value'), Input('region-filter', 'value')]
    )
    def update_dashboard(selected_branch, selected_region):
        # 관리지사별 데이터 필터링
        filtered_df = raw_df[raw_df['관리지사명'] == selected_branch] if selected_branch else raw_df

        # 영업구역별 데이터 필터링
        if selected_region:
            filtered_df = filtered_df[filtered_df['영업구역정보'] == selected_region]

        # 그래프 생성
        branch_summary = filtered_df.groupby('관리지사명').agg(
            계약번호_건수=('계약번호', 'count'),
            월정료합계=('KTT월정료', 'sum')
        ).reset_index()

        main_chart = px.bar(
            branch_summary,
            x='관리지사명',
            y='계약번호_건수',
            text='월정료합계',
            title="전체 관리지사 건수 및 월정료" if not selected_branch else f"{selected_branch} 건수 및 월정료",
            color='관리지사명'
        )
        main_chart.update_traces(texttemplate='%{text:.2s}천원', textposition='outside')

        # 현황표 업데이트
        table_data = filtered_df.groupby(['관리지사명', '영업구역정보']).agg(
            계약번호_건수=('계약번호', 'count'),
            KTT월정료=('KTT월정료', 'sum')
        ).reset_index().to_dict('records')

        # 영업구역 필터 옵션 업데이트
        region_options = [{'label': region, 'value': region} for region in filtered_df['영업구역정보'].unique()]

        return main_chart, table_data, region_options

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리 및 사용자 정의 정렬 적용
    raw_df, processed_summary_df = preprocess_data(raw_df)

    # Dash 대시보드 실행
    app = create_dash_app(processed_summary_df, raw_df)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import os
import random

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거 및 사용자 정의 정렬
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명']).agg(
        계약번호_건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['계약번호_건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return df, summary

# Dash 대시보드 생성
def create_dash_app(summary_df, raw_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
            html.Label("영업구역정보 필터:", style={'font-weight': 'bold', 'margin-top': '10px'}),
            dcc.Dropdown(
                id='region-filter',
                placeholder="영업구역명을 선택하세요",
                multi=False
            )
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='main-chart'),
        html.H2("현황표", style={'textAlign': 'center', 'margin-top': '20px'}),
        dash_table.DataTable(
            id='data-table',
            columns=[
                {"name": "관리지사명", "id": "관리지사명"},
                {"name": "영업구역정보", "id": "영업구역정보"},
                {"name": "계약번호_건수", "id": "계약번호_건수"},
                {"name": "KTT월정료 (천원)", "id": "KTT월정료", "type": "numeric", "format": {"specifier": ",.0f"}}
            ],
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
            style_data={'border': '1px solid grey'},
            page_size=10,
            style_as_list_view=True
        )
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 콜백: 메인 차트 및 테이블 업데이트
    @app.callback(
        [Output('main-chart', 'figure'),
         Output('data-table', 'data'),
         Output('region-filter', 'options')],
        [Input('branch-filter', 'value'), Input('region-filter', 'value')]
    )
    def update_dashboard(selected_branch, selected_region):
        # 관리지사별 데이터 필터링
        filtered_df = raw_df[raw_df['관리지사명'] == selected_branch] if selected_branch else raw_df

        # 영업구역별 데이터 필터링
        if selected_region:
            filtered_df = filtered_df[filtered_df['영업구역정보'] == selected_region]

        # 그래프 생성
        branch_summary = filtered_df.groupby('관리지사명').agg(
            계약번호_건수=('계약번호', 'count'),
            월정료합계=('KTT월정료', 'sum')
        ).reset_index()

        main_chart = px.bar(
            branch_summary,
            x='관리지사명',
            y='계약번호_건수',
            text='월정료합계',
            title="전체 관리지사 건수 및 월정료" if not selected_branch else f"{selected_branch} 건수 및 월정료",
            color='관리지사명'
        )
        main_chart.update_traces(texttemplate='%{text:.2s}천원', textposition='outside')
        main_chart.update_layout(
            height=500,  # 그래프 크기 조정
            margin=dict(t=50, l=50, r=50, b=50),  # 여백 조정
            title={'x': 0.5, 'xanchor': 'center'},
            xaxis=dict(tickangle=-45),
            font=dict(size=14)  # 글꼴 크기 조정
        )

        # 현황표 업데이트
        table_data = filtered_df.groupby(['관리지사명', '영업구역정보']).agg(
            계약번호_건수=('계약번호', 'count'),
            KTT월정료=('KTT월정료', 'sum')
        ).reset_index().to_dict('records')

        # 영업구역 필터 옵션 업데이트
        region_options = [{'label': region, 'value': region} for region in filtered_df['영업구역정보'].unique()]

        return main_chart, table_data, region_options

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리 및 사용자 정의 정렬 적용
    raw_df, processed_summary_df = preprocess_data(raw_df)

    # Dash 대시보드 실행
    app = create_dash_app(processed_summary_df, raw_df)

    # 동적으로 고유한 포트를 생성
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    app.run_server(debug=True, port=port)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import os
import random

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거 및 사용자 정의 정렬
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명', '영업구역정보']).agg(
        계약번호_건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['계약번호_건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return df, summary

# Dash 대시보드 생성
def create_dash_app(summary_df, raw_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='branch-chart'),
        html.H2("현황표", style={'textAlign': 'center', 'margin-top': '20px'}),
        dash_table.DataTable(
            id='data-table',
            columns=[
                {"name": "관리지사명", "id": "관리지사명"},
                {"name": "영업구역정보", "id": "영업구역정보"},
                {"name": "계약번호_건수", "id": "계약번호_건수"},
                {"name": "KTT월정료 (천원)", "id": "월정료합계", "type": "numeric", "format": {"specifier": ",.0f"}}
            ],
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
            style_data={'border': '1px solid grey'},
            page_size=10,
            style_as_list_view=True
        ),
        html.Div(id='port-info', style={'textAlign': 'center', 'margin-top': '20px'})
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 콜백: 지사별 차트 및 표 업데이트
    @app.callback(
        [Output('branch-chart', 'figure'), Output('data-table', 'data')],
        [Input('branch-filter', 'value')]
    )
    def update_dashboard(selected_branch):
        # 관리지사별 데이터 필터링
        filtered_df = summary_df[summary_df['관리지사명'] == selected_branch] if selected_branch else summary_df

        # 그래프 생성
        branch_chart = px.bar(
            filtered_df,
            x='영업구역정보',
            y='계약번호_건수',
            text='표시값',
            color='영업구역정보',
            title=f"{selected_branch} 영업구역정보별 건수 및 월정료" if selected_branch else "전체 영업구역정보별 건수 및 월정료"
        )
        branch_chart.update_traces(texttemplate='%{text}', textposition='outside')
        branch_chart.update_layout(
            height=500,  # 그래프 크기 조정
            margin=dict(t=50, l=50, r=50, b=50),  # 여백 조정
            title={'x': 0.5, 'xanchor': 'center'},
            xaxis=dict(tickangle=-45),
            font=dict(size=14)  # 글꼴 크기 조정
        )

        # 표 업데이트
        table_data = filtered_df[['관리지사명', '영업구역정보', '계약번호_건수', '월정료합계']].to_dict('records')

        return branch_chart, table_data

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리 및 사용자 정의 정렬 적용
    raw_df, processed_summary_df = preprocess_data(raw_df)

    # Dash 대시보드 실행
    app = create_dash_app(processed_summary_df, raw_df)

    # 동적으로 고유한 포트를 생성 및 표시
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    print(f"대시보드가 http://127.0.0.1:{port} 에서 실행 중입니다.")
    app.run_server(debug=True, port=port)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import os
import random

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거 및 사용자 정의 정렬
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명', '영업구역정보']).agg(
        계약번호_건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 건수 및 월정료 표시 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['계약번호_건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    return df, summary

# Dash 대시보드 생성
def create_dash_app(summary_df, raw_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in summary_df['관리지사명'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
        ], style={'margin-bottom': '20px'}),
        dcc.Graph(id='branch-chart'),
        html.Div(id='summary-section', style={'textAlign': 'center', 'margin-top': '20px'}),
        html.H2("현황표", style={'textAlign': 'center', 'margin-top': '20px'}),
        dash_table.DataTable(
            id='data-table',
            columns=[
                {"name": "관리지사명", "id": "관리지사명"},
                {"name": "영업구역정보", "id": "영업구역정보"},
                {"name": "계약번호_건수", "id": "계약번호_건수"},
                {"name": "KTT월정료 (천원)", "id": "월정료합계", "type": "numeric", "format": {"specifier": ",.0f"}}
            ],
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
            style_data={'border': '1px solid grey'},
            page_size=10,
            style_as_list_view=True
        ),
        html.Div(id='port-info', style={'textAlign': 'center', 'margin-top': '20px'})
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 콜백: 지사별 차트 및 표 업데이트
    @app.callback(
        [Output('branch-chart', 'figure'),
         Output('data-table', 'data'),
         Output('summary-section', 'children')],
        [Input('branch-filter', 'value')]
    )
    def update_dashboard(selected_branch):
        # 관리지사별 데이터 필터링
        filtered_df = summary_df[summary_df['관리지사명'] == selected_branch] if selected_branch else summary_df

        # 그래프 생성
        branch_chart = px.bar(
            filtered_df,
            x='영업구역정보',
            y='계약번호_건수',
            text='표시값',
            color='영업구역정보',
            title=f"{selected_branch} 영업구역정보별 건수 및 월정료" if selected_branch else "전체 영업구역정보별 건수 및 월정료"
        )
        branch_chart.update_traces(texttemplate='%{text}', textposition='outside')
        branch_chart.update_layout(
            height=500,  # 그래프 크기 조정
            margin=dict(t=50, l=50, r=50, b=50),  # 여백 조정
            title={'x': 0.5, 'xanchor': 'center'},
            xaxis=dict(tickangle=-45),
            font=dict(size=14)  # 글꼴 크기 조정
        )

        # 현황표 업데이트
        table_data = filtered_df[['관리지사명', '영업구역정보', '계약번호_건수', '월정료합계']].to_dict('records')

        # 요약 데이터 (건수 합계, 월정료 합계 및 점유율 계산)
        total_count = filtered_df['계약번호_건수'].sum()
        total_revenue = filtered_df['월정료합계'].sum()
        total_percentage = (filtered_df['계약번호_건수'].sum() / summary_df['계약번호_건수'].sum()) * 100

        summary_text = f"""
            선택된 지사의 총 건수: {total_count}건<br>
            선택된 지사의 총 월정료: {int(total_revenue):,}천원<br>
            점유율: {total_percentage:.2f}%
        """

        return branch_chart, table_data, html.Div(summary_text, style={'font-size': '18px', 'color': 'darkblue'})

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리 및 사용자 정의 정렬 적용
    raw_df, processed_summary_df = preprocess_data(raw_df)

    # Dash 대시보드 실행
    app = create_dash_app(processed_summary_df, raw_df)

    # 동적으로 고유한 포트를 생성 및 표시
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    print(f"대시보드가 http://127.0.0.1:{port} 에서 실행 중입니다.")
    app.run_server(debug=True, port=port)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import os
import random

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거 및 사용자 정의 정렬
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명', '영업구역정보']).agg(
        계약번호_건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 지사별 요약 데이터 생성
    branch_summary = df.groupby('관리지사명').agg(
        계약번호_건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['계약번호_건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    branch_summary['표시값'] = branch_summary.apply(
        lambda row: f"{row['계약번호_건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )

    return df, summary, branch_summary

# Dash 대시보드 생성
def create_dash_app(summary_df, branch_summary, raw_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),

        # 상단 지사별 시각화
        html.Div([
            dcc.Graph(
                id='overall-branch-chart',
                figure=px.bar(
                    branch_summary,
                    x='관리지사명',
                    y='계약번호_건수',
                    text='표시값',
                    color='관리지사명',
                    title="전체 관리지사별 건수 및 월정료",
                ).update_traces(texttemplate='%{text}', textposition='outside').update_layout(
                    height=500,
                    margin=dict(t=50, l=50, r=50, b=50),
                    title={'x': 0.5, 'xanchor': 'center'},
                    xaxis=dict(tickangle=-45),
                    font=dict(size=14)
                )
            )
        ], style={'margin-bottom': '20px'}),

        # 지사 선택 필터
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
        ], style={'margin-bottom': '20px'}),

        # 영업구역별 시각화
        dcc.Graph(id='branch-chart'),

        # 현황표 및 합계 정보
        html.Div(id='summary-section', style={'textAlign': 'center', 'margin-top': '20px'}),
        html.H2("현황표", style={'textAlign': 'center', 'margin-top': '20px'}),
        dash_table.DataTable(
            id='data-table',
            columns=[
                {"name": "관리지사명", "id": "관리지사명"},
                {"name": "영업구역정보", "id": "영업구역정보"},
                {"name": "계약번호_건수", "id": "계약번호_건수"},
                {"name": "KTT월정료 (천원)", "id": "월정료합계", "type": "numeric", "format": {"specifier": ",.0f"}}
            ],
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
            style_data={'border': '1px solid grey'},
            page_size=10,
            style_as_list_view=True
        ),
        html.Div(id='port-info', style={'textAlign': 'center', 'margin-top': '20px'})
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 콜백: 지사별 차트 및 표 업데이트
    @app.callback(
        [Output('branch-chart', 'figure'),
         Output('data-table', 'data'),
         Output('summary-section', 'children')],
        [Input('branch-filter', 'value')]
    )
    def update_dashboard(selected_branch):
        # 관리지사별 데이터 필터링
        filtered_df = summary_df[summary_df['관리지사명'] == selected_branch] if selected_branch else summary_df

        # 그래프 생성
        branch_chart = px.bar(
            filtered_df,
            x='영업구역정보',
            y='계약번호_건수',
            text='표시값',
            color='영업구역정보',
            title=f"{selected_branch} 영업구역정보별 건수 및 월정료" if selected_branch else "전체 영업구역정보별 건수 및 월정료"
        )
        branch_chart.update_traces(texttemplate='%{text}', textposition='outside')
        branch_chart.update_layout(
            height=500,
            margin=dict(t=50, l=50, r=50, b=50),
            title={'x': 0.5, 'xanchor': 'center'},
            xaxis=dict(tickangle=-45),
            font=dict(size=14)
        )

        # 현황표 업데이트
        table_data = filtered_df[['관리지사명', '영업구역정보', '계약번호_건수', '월정료합계']].to_dict('records')

        # 요약 데이터 (건수 합계, 월정료 합계 및 점유율 계산)
        total_count = filtered_df['계약번호_건수'].sum()
        total_revenue = filtered_df['월정료합계'].sum()
        total_percentage = (filtered_df['계약번호_건수'].sum() / summary_df['계약번호_건수'].sum()) * 100

        summary_text = f"""
            선택된 지사의 총 건수: {total_count}건<br>
            선택된 지사의 총 월정료: {int(total_revenue):,}천원<br>
            점유율: {total_percentage:.2f}%
        """

        return branch_chart, table_data, html.Div(summary_text, style={'font-size': '18px', 'color': 'darkblue'})

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리 및 사용자 정의 정렬 적용
    raw_df, processed_summary_df, processed_branch_summary = preprocess_data(raw_df)

    # Dash 대시보드 실행
    app = create_dash_app(processed_summary_df, processed_branch_summary, raw_df)

    # 동적으로 고유한 포트를 생성 및 표시
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    print(f"대시보드가 http://127.0.0.1:{port} 에서 실행 중입니다.")
    app.run_server(debug=True, port=port)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import os
import random

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)  # "db" 시트 가져오기
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# 데이터 전처리
def preprocess_data(df):
    # KTT월정료: 문자열에서 숫자로 변환
    df['KTT월정료'] = df['KTT월정료'].replace({'₩': '', ',': ''}, regex=True).astype(float)

    # "지사" 텍스트 제거 및 사용자 정의 정렬
    custom_order = ['중앙', '강북', '서대문', '고양', '의정부', '남양주', '강릉', '원주']
    df['관리지사명_수정'] = df['관리지사명'].str.replace('지사', '', regex=False)
    df['관리지사명_수정'] = pd.Categorical(df['관리지사명_수정'], categories=custom_order, ordered=True)
    df = df.sort_values('관리지사명_수정')

    # 요약 데이터 생성
    summary = df.groupby(['관리지사명', '영업구역정보']).agg(
        계약번호_건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 지사별 요약 데이터 생성
    branch_summary = df.groupby('관리지사명').agg(
        계약번호_건수=('계약번호', 'count'),
        월정료합계=('KTT월정료', 'sum')
    ).reset_index()

    # 텍스트 생성
    summary['표시값'] = summary.apply(
        lambda row: f"{row['계약번호_건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )
    branch_summary['표시값'] = branch_summary.apply(
        lambda row: f"{row['계약번호_건수']}건\n{int(row['월정료합계']):,}천원", axis=1
    )

    return df, summary, branch_summary

# Dash 대시보드 생성
def create_dash_app(summary_df, branch_summary, raw_df):
    app = Dash(__name__, suppress_callback_exceptions=True)

    # 레이아웃 정의
    app.layout = html.Div([
        html.H1("정지분석 대시보드", style={'textAlign': 'center'}),

        # 상단 지사별 시각화
        html.Div([
            dcc.Graph(
                id='overall-branch-chart',
                figure=px.bar(
                    branch_summary,
                    x='관리지사명',
                    y='계약번호_건수',
                    text='표시값',
                    color='관리지사명',
                    title="전체 관리지사별 건수 및 월정료",
                ).update_traces(texttemplate='%{text}', textposition='outside').update_layout(
                    height=500,
                    margin=dict(t=50, l=50, r=50, b=50),
                    title={'x': 0.5, 'xanchor': 'center'},
                    xaxis=dict(tickangle=-45),
                    font=dict(size=14)
                )
            )
        ], style={'margin-bottom': '20px'}),

        # 지사 선택 필터
        html.Div([
            html.Label("관리지사명 필터:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='branch-filter',
                options=[{'label': branch, 'value': branch} for branch in branch_summary['관리지사명'].unique()],
                value=None,
                placeholder="관리지사를 선택하세요",
                multi=False
            ),
        ], style={'margin-bottom': '20px'}),

        # 영업구역별 시각화
        dcc.Graph(id='branch-chart'),

        # 현황표 및 합계 정보
        html.Div(id='summary-section', style={'textAlign': 'center', 'margin-top': '20px'}),
        html.H2("현황표", style={'textAlign': 'center', 'margin-top': '20px'}),
        dash_table.DataTable(
            id='data-table',
            columns=[
                {"name": "관리지사명", "id": "관리지사명"},
                {"name": "영업구역정보", "id": "영업구역정보"},
                {"name": "계약번호_건수", "id": "계약번호_건수"},
                {"name": "KTT월정료 (천원)", "id": "월정료합계", "type": "numeric", "format": {"specifier": ",.0f"}}
            ],
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
            style_data={'border': '1px solid grey'},
            page_size=10,
            style_as_list_view=True
        ),
        html.Div(id='port-info', style={'textAlign': 'center', 'margin-top': '20px'})
    ], style={'padding': '20px', 'max-width': '1200px', 'margin': '0 auto'})

    # 콜백: 지사별 차트 및 표 업데이트
    @app.callback(
        [Output('branch-chart', 'figure'),
         Output('data-table', 'data'),
         Output('summary-section', 'children')],
        [Input('branch-filter', 'value')]
    )
    def update_dashboard(selected_branch):
        # 관리지사별 데이터 필터링
        filtered_df = summary_df[summary_df['관리지사명'] == selected_branch] if selected_branch else summary_df

        # 그래프 생성
        branch_chart = px.bar(
            filtered_df,
            x='영업구역정보',
            y='계약번호_건수',
            text='표시값',
            color='영업구역정보',
            title=f"{selected_branch} 영업구역정보별 건수 및 월정료" if selected_branch else "전체 영업구역정보별 건수 및 월정료"
        )
        branch_chart.update_traces(texttemplate='%{text}', textposition='outside')
        branch_chart.update_layout(
            height=500,
            margin=dict(t=50, l=50, r=50, b=50),
            title={'x': 0.5, 'xanchor': 'center'},
            xaxis=dict(tickangle=-45),
            font=dict(size=14)
        )

        # 현황표 업데이트
        table_data = filtered_df[['관리지사명', '영업구역정보', '계약번호_건수', '월정료합계']].to_dict('records')

        # 요약 데이터 (건수 합계, 월정료 합계 및 점유율 계산)
        total_count = filtered_df['계약번호_건수'].sum()
        total_revenue = filtered_df['월정료합계'].sum()
        total_percentage = (filtered_df['계약번호_건수'].sum() / summary_df['계약번호_건수'].sum()) * 100

        summary_text = f"""
            선택된 지사의 총 건수: {total_count}건<br>
            선택된 지사의 총 월정료: {int(total_revenue):,}천원<br>
            점유율: {total_percentage:.2f}%
        """

        return branch_chart, table_data, html.Div(summary_text, style={'font-size': '18px', 'color': 'darkblue'})

    return app

# 실행
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive 마운트

    # Google Sheets URL 및 시트 이름 설정
    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"
    sheet_name = "db"  # "db" 시트 이름

    # Google Sheets API 인증
    client = authenticate_google_sheets()

    # Google Sheets 데이터 가져오기
    raw_df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

    # 데이터 전처리 및 사용자 정의 정렬 적용
    raw_df, processed_summary_df, processed_branch_summary = preprocess_data(raw_df)

    # Dash 대시보드 실행
    app = create_dash_app(processed_summary_df, processed_branch_summary, raw_df)

    # 동적으로 고유한 포트를 생성 및 표시
    port = int(os.environ.get("PORT", random.randint(1024, 65535)))
    print(f"대시보드가 http://127.0.0.1:{port} 에서 실행 중입니다.")
    app.run_server(debug=True, port=port)

In [ ]:
!pkill -f ngrok

In [ ]:
!ngrok config add-authtoken 2hrbKVWli8EAuPYqh1roetTXNUZ_6mzp6SbmMuMprYrV52eS9

In [ ]:
from pyngrok import ngrok

# ngrok 포트 연결 (HTTP 전용으로 설정)
port = 8050
public_url = ngrok.connect(port, "http")
print(f"대시보드가 {public_url} 에서 실행 중입니다.")
app.run_server(debug=True, port=port)

In [ ]:
from pyngrok import ngrok
import os
from flask import Flask
from dash import Dash

server = Flask(__name__)
app = Dash(__name__, server=server)

# Dash 레이아웃 정의
app.layout = html.Div([
    html.H1("Dash 대시보드"),
    html.P("ngrok를 통해 외부에서 접속할 수 있습니다!")
])

# ngrok 연결 및 Dash 실행
port = 8050
public_url = ngrok.connect(port, "http")
print(f"대시보드가 {public_url} 에서 실행 중입니다.")
app.run_server(debug=True, port=port, use_reloader=False)

In [ ]:
from pyngrok import ngrok

port = 8050
public_url = ngrok.connect(port, "http")
print(f"대시보드가 {public_url} 에서 실행 중입니다.")
app.run_server(debug=True, port=port, use_reloader=False)

In [ ]:
!pkill -f ngrok

In [ ]:
from pyngrok import ngrok

# ngrok 포트 연결
port = 8050
public_url = ngrok.connect(port, "http")
print(f"대시보드가 {public_url} 에서 실행 중입니다.")
app.run_server(debug=True, port=port, use_reloader=False)

In [ ]:
app.run_server(debug=True, port=8050)

In [ ]:
# 중복 실행 방지 코드
if __name__ == "__main__":
    try:
        app.run_server(debug=True, port=port, use_reloader=False)
    except AssertionError as e:
        print(f"오류 발생: {e}. 서버가 이미 실행 중일 수 있습니다.")

In [ ]:
!pkill -f dash

In [ ]:
# prompt: !pkill -f ngrok

!pkill -f ngrok

In [ ]:
!pkill -f ngrok
!pkill -f dash

In [ ]:
from pyngrok import ngrok
from dash import Dash, html

# Dash 앱 생성
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Dash 대시보드"),
    html.P("ngrok를 통해 외부에서 접속 가능합니다!")
])

# ngrok 포트 연결
port = 8050
public_url = ngrok.connect(port, "http")
print(f"대시보드가 {public_url} 에서 실행 중입니다.")

# Dash 실행
if __name__ == "__main__":
    app.run_server(debug=True, port=port, use_reloader=False)

In [ ]:
!pip install dash pyngrok

In [ ]:
!pip install gspread pandas matplotlib seaborn ipywidgets oauth2client

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from ipywidgets import widgets, interact

# 한글 폰트 설치 및 설정
!sudo apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib

# 한글 폰트 설정 함수
def set_korean_fonts():
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    font_prop = fm.FontProperties(fname=font_path)
    plt.rc('font', family=font_prop.get_name())  # 기본 폰트 설정
    plt.rcParams['font.family'] = font_prop.get_name()  # 기본 폰트 설정
    plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
    return font_prop

# 폰트 적용
font_prop = set_korean_fonts()

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# Google Sheets 데이터 로드
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"  # 스프레드시트 URL
sheet_name = "db"  # 시트 이름
client = authenticate_google_sheets()
df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

# 데이터 확인
print("데이터 로드 완료. 데이터 프레임 크기:", df.shape)
print(df.head())

# 스타일 설정
sns.set_style("whitegrid")
palette = sns.color_palette("viridis")

# 열 헤더 선택 인터페이스
column_options = list(df.columns)
selected_columns = widgets.SelectMultiple(
    options=column_options,
    value=[column_options[0]],
    description='열 선택:',
    disabled=False
)

# 분포 시각화 함수
def visualize_columns(selected_columns):
    if not selected_columns:
        print("열을 선택하세요.")
        return

    # 선택된 열 분포 시각화
    for column in selected_columns:
        plt.figure(figsize=(12, 8))
        if pd.api.types.is_numeric_dtype(df[column]):
            sns.histplot(df[column], kde=True, bins=30, color='skyblue')
            plt.title(f"'{column}' 분포", fontsize=18, fontproperties=font_prop, weight='bold')
            plt.xlabel(column, fontsize=14, fontproperties=font_prop, weight='bold')
            plt.ylabel('빈도', fontsize=14, fontproperties=font_prop, weight='bold')
        else:
            ax = sns.countplot(y=df[column], palette=palette)
            plt.title(f"'{column}' 카테고리 분포", fontsize=18, fontproperties=font_prop, weight='bold')
            plt.xlabel('빈도', fontsize=14, fontproperties=font_prop, weight='bold')
            plt.ylabel(column, fontsize=14, fontproperties=font_prop, weight='bold')

            # 막대 내부에 건수와 점유율 표시
            total = len(df[column])  # 총 데이터 개수
            for p in ax.patches:
                count = int(p.get_width())
                percentage = 100 * count / total
                ax.annotate(f'{count}\n({percentage:.1f}%)',
                            (p.get_width(), p.get_y() + p.get_height() / 2),
                            ha='left', va='center', fontsize=10, fontproperties=font_prop)

            # Y축 레이블 폰트 강제 적용
            for label in ax.get_yticklabels():
                label.set_fontproperties(font_prop)

        plt.legend(labels=['데이터 분포'], prop=font_prop, fontsize=12, loc='best')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

# 상관관계 히트맵 함수
def plot_correlation():
    # 상관관계 분석에 사용할 수치형 데이터만 선택
    numeric_df = df.select_dtypes(include=['number'])
    if numeric_df.empty:
        print("상관관계를 분석할 수치형 데이터가 없습니다.")
        return

    plt.figure(figsize=(14, 10))
    ax = sns.heatmap(numeric_df.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, linecolor='gray')
    plt.title("상관관계 히트맵", fontsize=18, fontproperties=font_prop, weight='bold')
    plt.xlabel('특성', fontsize=14, fontproperties=font_prop, weight='bold')
    plt.ylabel('특성', fontsize=14, fontproperties=font_prop, weight='bold')

    # X축, Y축 폰트 강제 적용
    for label in ax.get_xticklabels():
        label.set_fontproperties(font_prop)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)

    plt.show()

# EDA 대화형 UI
def run_eda(selected_columns):
    visualize_columns(selected_columns)
    plot_correlation()

interact(run_eda, selected_columns=selected_columns)

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# 한글 폰트 설치 및 설정
!sudo apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib

# 한글 폰트 설정 함수
def set_korean_fonts():
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    font_prop = fm.FontProperties(fname=font_path)
    plt.rc('font', family=font_prop.get_name())  # 기본 폰트 설정
    plt.rcParams['font.family'] = font_prop.get_name()  # 기본 폰트 설정
    plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
    return font_prop

# 폰트 적용
font_prop = set_korean_fonts()

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# Google Sheets 데이터 로드
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"  # 스프레드시트 URL
sheet_name = "db"  # 시트 이름
client = authenticate_google_sheets()
df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

# 데이터 확인
print("데이터 로드 완료. 데이터 프레임 크기:", df.shape)
print(df.head())

# 건수를 계약번호 행 개수로 설정
df['건수'] = df.groupby('관리지사명')['계약번호'].transform('count')

# 데이터 전처리: 관리지사명, 건수, KTT월정료 선택
df_visual = df[['관리지사명', '건수', 'KTT월정료']].drop_duplicates()

# 데이터 확인
print(df_visual.head())

# 시각화 함수
def visualize_data():
    # 데이터 정렬 (관리지사명을 기준으로)
    df_sorted = df_visual.sort_values(by='관리지사명')

    plt.figure(figsize=(14, 8))

    # 첫 번째 Y축: 건수 (막대 그래프)
    ax1 = plt.gca()
    bars = ax1.bar(df_sorted['관리지사명'], df_sorted['건수'], color='skyblue', alpha=0.8, label='건수')
    ax1.set_ylabel('건수', fontsize=14, fontproperties=font_prop)
    ax1.tick_params(axis='y', labelsize=12)
    ax1.set_xlabel('관리지사명', fontsize=14, fontproperties=font_prop)
    ax1.tick_params(axis='x', labelrotation=45, labelsize=12)

    # 막대 내부에 건수 표시
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width() / 2, height, f'{int(height)}',
                 ha='center', va='bottom', fontsize=10, fontproperties=font_prop)

    # 두 번째 Y축: KTT월정료 (선 그래프)
    ax2 = ax1.twinx()
    ax2.plot(df_sorted['관리지사명'], df_sorted['KTT월정료'], color='red', marker='o', label='KTT월정료', linewidth=2)
    ax2.set_ylabel('KTT월정료', fontsize=14, fontproperties=font_prop)
    ax2.tick_params(axis='y', labelsize=12)

    # 제목 설정
    plt.title('관리지사명별 건수 및 KTT월정료', fontsize=16, fontproperties=font_prop, weight='bold')

    # 범례 설정
    ax1.legend(loc='upper left', fontsize=12, prop=font_prop)
    ax2.legend(loc='upper right', fontsize=12, prop=font_prop)

    plt.tight_layout()
    plt.show()

# 시각화 실행
visualize_data()

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# 한글 폰트 설치 및 설정
!sudo apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib

# 한글 폰트 설정 함수
def set_korean_fonts():
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    font_prop = fm.FontProperties(fname=font_path)
    plt.rc('font', family=font_prop.get_name())  # 기본 폰트 설정
    plt.rcParams['font.family'] = font_prop.get_name()  # 기본 폰트 설정
    plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
    return font_prop

# 폰트 적용
font_prop = set_korean_fonts()

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# Google Sheets 데이터 로드
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"  # 스프레드시트 URL
sheet_name = "db"  # 시트 이름
client = authenticate_google_sheets()
df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

# 데이터 확인
print("데이터 로드 완료. 데이터 프레임 크기:", df.shape)
print(df.head())

# 건수를 계약번호 행 개수로 설정
df['건수'] = df.groupby('관리지사명')['계약번호'].transform('count')

# KTT월정료 열을 숫자형으로 변환
df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce')

# NaN 값 확인 후 제거
print("KTT월정료 변환 후 NaN 개수:", df['KTT월정료'].isna().sum())
df = df.dropna(subset=['KTT월정료'])

# 데이터 전처리: 관리지사명, 건수, KTT월정료
df_visual = df[['관리지사명', '건수', 'KTT월정료']].drop_duplicates()

# 데이터 검증
print("전처리 완료. 데이터 프레임 크기:", df_visual.shape)
print(df_visual.head())

# 시각화 함수
def visualize_data():
    # 데이터 정렬 (관리지사명을 기준으로)
    df_sorted = df_visual.sort_values(by='관리지사명')

    plt.figure(figsize=(14, 8))

    # 첫 번째 Y축: 건수 (막대 그래프)
    ax1 = plt.gca()
    bars = ax1.bar(df_sorted['관리지사명'], df_sorted['건수'], color='skyblue', alpha=0.8, label='건수')
    ax1.set_ylabel('건수', fontsize=14, fontproperties=font_prop)
    ax1.tick_params(axis='y', labelsize=12)
    ax1.set_xlabel('관리지사명', fontsize=14, fontproperties=font_prop)
    ax1.tick_params(axis='x', labelrotation=45, labelsize=12)

    # 막대 내부에 건수 표시
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width() / 2, height, f'{int(height)}',
                 ha='center', va='bottom', fontsize=10, fontproperties=font_prop)

    # 두 번째 Y축: 월정료 (선 그래프)
    ax2 = ax1.twinx()
    ax2.plot(df_sorted['관리지사명'], df_sorted['KTT월정료'], color='red', marker='o', label='KTT월정료', linewidth=2)
    ax2.set_ylabel('KTT월정료', fontsize=14, fontproperties=font_prop)
    ax2.tick_params(axis='y', labelsize=12)

    # 제목 설정
    plt.title('관리지사명별 건수 및 월정료', fontsize=16, fontproperties=font_prop, weight='bold')

    # 범례 설정
    ax1.legend(loc='upper left', fontsize=12, prop=font_prop)
    ax2.legend(loc='upper right', fontsize=12, prop=font_prop)

    # 배경 스타일 설정
    plt.grid(axis='both', linestyle='--', linewidth=0.5, alpha=0.7)
    sns.set_style("whitegrid")

    plt.tight_layout()
    plt.show()

# 시각화 실행
visualize_data()

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# 한글 폰트 설치 및 설정
!sudo apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib

# 한글 폰트 설정 함수
def set_korean_fonts():
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    font_prop = fm.FontProperties(fname=font_path)
    plt.rc('font', family=font_prop.get_name())  # 기본 폰트 설정
    plt.rcParams['font.family'] = font_prop.get_name()  # 기본 폰트 설정
    plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
    return font_prop

# 폰트 적용
font_prop = set_korean_fonts()

# Google Sheets API 인증
def authenticate_google_sheets():
    creds_path = '/content/drive/MyDrive/Key/credentials.json'  # Google Drive Key 경로
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
    client = gspread.authorize(creds)
    return client

# Google Sheets 데이터 가져오기
def fetch_google_sheet_data(spreadsheet_url, sheet_name, client):
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

# Google Sheets 데이터 로드
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0/edit?usp=sharing"  # 스프레드시트 URL
sheet_name = "db"  # 시트 이름
client = authenticate_google_sheets()
df = fetch_google_sheet_data(spreadsheet_url, sheet_name, client)

# 데이터 확인
print("데이터 로드 완료. 데이터 프레임 크기:", df.shape)
print(df.head())

# 건수를 계약번호 행 개수로 설정
df['건수'] = df.groupby('관리지사명')['계약번호'].transform('count')

# KTT월정료 열을 숫자형으로 변환
df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce')

# NaN 값 확인 후 제거
print("KTT월정료 변환 후 NaN 개수:", df['KTT월정료'].isna().sum())
df = df.dropna(subset=['KTT월정료'])

# 데이터 전처리: 관리지사명, 건수, KTT월정료
df_visual = df[['관리지사명', '건수', 'KTT월정료']].drop_duplicates()

# 데이터 검증
print("전처리 완료. 데이터 프레임 크기:", df_visual.shape)
print(df_visual.head())

# 건수 및 KTT월정료 값 확인
print("건수 값 확인:", df_visual['건수'].unique())
print("KTT월정료 값 확인:", df_visual['KTT월정료'].unique())

# 시각화 함수
def visualize_data():
    # 데이터 정렬 (관리지사명을 기준으로)
    df_sorted = df_visual.sort_values(by='관리지사명')

    plt.figure(figsize=(14, 8))

    # 첫 번째 Y축: 건수 (막대 그래프)
    ax1 = plt.gca()
    bars = ax1.bar(df_sorted['관리지사명'], df_sorted['건수'], color='skyblue', alpha=0.8, label='건수')
    ax1.set_ylabel('건수', fontsize=14, fontproperties=font_prop)
    ax1.tick_params(axis='y', labelsize=12)
    ax1.set_xlabel('관리지사명', fontsize=14, fontproperties=font_prop)
    ax1.tick_params(axis='x', labelrotation=45, labelsize=12)

    # 막대 내부에 건수 표시
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width() / 2, height, f'{int(height)}',
                 ha='center', va='bottom', fontsize=10, fontproperties=font_prop)

    # 두 번째 Y축: 월정료 (선 그래프)
    ax2 = ax1.twinx()
    ax2.plot(df_sorted['관리지사명'], df_sorted['KTT월정료'], color='red', marker='o', label='KTT월정료', linewidth=2)
    ax2.set_ylabel('KTT월정료', fontsize=14, fontproperties=font_prop)
    ax2.tick_params(axis='y', labelsize=12)

    # 제목 설정
    plt.title('관리지사명별 건수 및 월정료', fontsize=16, fontproperties=font_prop, weight='bold')

    # 범례 설정
    ax1.legend(loc='upper left', fontsize=12, prop=font_prop)
    ax2.legend(loc='upper right', fontsize=12, prop=font_prop)

    # 배경 스타일 설정
    plt.grid(axis='both', linestyle='--', linewidth=0.5, alpha=0.7)
    sns.set_style("whitegrid")

    plt.tight_layout()
    plt.show()

# 시각화 실행
visualize_data()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (40.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 124561 files and direc

GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
# 📊 시트 데이터 가져오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)

# 🔍 설치주소 컬럼 자동 탐색 (보통 '설치주소' 또는 R열)
address_col = '설치주소' if '설치주소' in df.columns else df.columns[17]
df['주소_원본'] = df[address_col]

# 🧩 시군구면리 분해 함수
def split_korean_address(addr):
    addr = str(addr)
    parts = re.findall(r'([가-힣]+[시군구]|[가-힣]+[읍면동리])', addr)
    return pd.Series(parts[:5] + ['']*(5 - len(parts)), index=['시', '군구', '읍면', '동리', '기타'])

# 📥 분해 컬럼 생성
split_df = df['주소_원본'].apply(split_korean_address)
df = pd.concat([df, split_df], axis=1)

# 📤 결과를 다시 시트에 저장 (기존 덮어쓰기 주의)
set_with_dataframe(worksheet, df)

print("✅ 설치주소 → 시군구면리 컬럼 추가 완료!")
